In [408]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [409]:
def delta(t1,t2):
    return((t2-t1).days/360)

class DISC_Curve:
    def __init__(self,_spot_date,_zeros,_mat):
        self.spot_date=_spot_date
        self.zeros=_zeros
        self.mat=_mat
        self.tenors= [delta(spot_date,i) for i in _mat]
        self.libors=[((1/pv)-1)/t for (pv,t) in zip(_zeros,self.tenors)]
        
    def get_zero(self,maturity):
        t=delta(self.spot_date,maturity)
        l=np.interp(t,self.tenors,self.libors)
        return(1/(1+l*t))
    
    def get_libor(self,maturity):
        t=delta(self.spot_date,maturity)
        return(np.interp(t,self.tenors,self.libors))
    
    def set_node(self,zero, mat_date):
        self.zeros.append(zero)
        self.mat.append(mat_date)
        tenor=delta(self.spot_date,mat_date)
        self.tenors.append(tenor)
        self.libors.append(((1/zero)-1)/tenor)
    
    def get_fwd(self,reset_date,maturity_date):
        return(((self.get_zero(reset_date)/self.get_zero(maturity_date))-1)/delta(reset_date,maturity_date))
        

In [410]:
mkt_data=pd.read_csv('C:/Users/claud/OneDrive/Escritorio/prueba.csv')
pmts=pd.read_csv('C:/Users/claud/OneDrive/Escritorio/pagos instrumentos.csv')
spot_date=pd.to_datetime('03/10/2012',format='%d/%m/%Y')
mkt_data['Maturity Dates']=pd.to_datetime(mkt_data['Maturity Dates'],format='%d/%m/%Y')
pmts.Tenor=pd.to_datetime(pmts.Tenor,format='%d/%m/%Y')

def delta(t1,t2):
    return((t2-t1).days/360)

In [411]:
cash_flows=pd.DataFrame(0.0,index=mkt_data['Maturity Dates'],columns=pmts.Tenor)
LIBORS=mkt_data.loc[mkt_data.Source=="LIBOR"]
FUTURES=mkt_data.loc[mkt_data.Source=="Futures"]
SWAPS=mkt_data.loc[mkt_data.Source=="Swap"]

p=np.matrix([1 if (i =='Swap') or (i =='LIBOR') else 0 for i in mkt_data.Source])

S=list(pmts.Tenor.loc[pmts.Instrument=='FUT'])
U=list(pmts.Tenor.loc[pmts.Instrument=='SWAP'])
U.insert(0,spot_date)

SWAP_DATES=list(SWAPS['Maturity Dates'])

for i in range(len(LIBORS)):
    v=LIBORS['Market Quotes'].iloc[i]/100
    t=list(LIBORS['Maturity Dates'])[i]
    cash_flows[t].loc[t]=1+v*delta(spot_date,t)
    
for i in range(1,len(S)):
    v=1-FUTURES['Market Quotes'].iloc[i-1]/100
    cash_flows[S[i-1]].loc[S[i]]=-1
    cash_flows[S[i]].loc[S[i]]=1+v*delta(S[i-1],S[i])
    
for i in range(len(SWAP_DATES)):
    v=SWAPS['Market Quotes'].iloc[i]
    for j in range(len(U)-1):
        if U[j+1]<SWAP_DATES[i]:
            cash_flows[U[j+1]].loc[SWAP_DATES[i]]=v*delta(U[j],U[j+1])/100
        if U[j+1] == SWAP_DATES[i]:
            cash_flows[U[j+1]].loc[SWAP_DATES[i]]=1+v*delta(U[j],U[j+1])/100

In [446]:
x=np.zeros((np.size(C[0,]),1))
x[0]=1

C=np.matrix(cash_flows)

l=list(cash_flows.columns)
l.insert(0,spot_date)
W=pd.Series(l).diff()
W=np.diag(list(W[1:].apply(lambda x: 1/np.sqrt(x.days/360))))
M=np.zeros((np.size(C[0,]),np.size(C[0,])))
M[0,0]=1
for i in range(1,len(M)):
    M[i,i]=1
    M[i,i-1]=-1
    
A=np.matmul(C,np.matmul(np.linalg.inv(M),np.linalg.inv(W)))

H=np.matmul(np.transpose(A),np.linalg.inv(np.matmul(A,np.transpose(A))))

G=np.matmul(C,np.linalg.inv(M))
R=np.matmul(G,x)
R=[R[i,0] for i in range(len (R))]

MM=p-R
                            
Delta=np.matmul(H,np.transpose(MM))
                
DF=pd.DataFrame(np.matmul(np.linalg.inv(M),np.matmul(np.linalg.inv(W),Delta)+x),index=pmts.Tenor)

t=delta(DF.index[len(DF.index)-2],DF.index[len(DF.index)-1])
100*(DF.loc[DF.index[len(DF.index)-2]]/DF.loc[DF.index[len(DF.index)-1]]-1)/t

0    2.531068
dtype: float64